In [1]:
import pandas as pd
import geopy.distance
import problem
from merge_transformer import MergeTransformer
from create_db import create_db
import os

In [2]:
X_train, y_train = problem.get_train_data('..')
X_train.loc[:, "DateOfDeparture"] = pd.to_datetime(X_train['DateOfDeparture'])

X_test, y_test = problem.get_test_data('..')
X_test.loc[:, "DateOfDeparture"] = pd.to_datetime(X_test['DateOfDeparture'])

In [3]:
def merge_dfs(database, X):
    
    X['Days_to_departure'] = (X['WeeksToDeparture'] * 7).round()
    X['DateBooked'] = X['DateOfDeparture'] -  pd.to_timedelta(X['Days_to_departure'], unit='d')
    
    merge_transform = MergeTransformer(X_ext=database['Date'], how='left', on=['DateOfDeparture'])
    X = merge_transform.fit_transform(X)
    
    print(database['Airport'].shape)
    print(database['AirportStatistics'].shape)
    print(database['StateFeatures'].shape)
        
    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'Departure',
                        'latitude_deg': 'latitude_departure',
                        'longitude_deg': 'longitude_departure',
                        'state': 'state_departure',
                        'capacity': 'capacity_departure',
                        'pop2010': 'population_departure'},
        how='left', on=['Departure'])
    X = merge_transform.fit_transform(X)
    
    print("departure airport = ", X.shape)

    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'Arrival', 
                        'latitude_deg': 'latitude_arrival', 
                        'longitude_deg': 'longitude_arrival',
                        'state': 'state_arrival',
                        'capacity': 'capacity_arrival',
                        'pop2010': 'population_arrival'},
        how='left', on=['Arrival'])

    X = merge_transform.fit_transform(X)
    
    print("arrival airport = ", X.shape)
    
    X['distance'] = X.apply(lambda x: geopy.distance.geodesic(
        (x.latitude_departure, x.longitude_departure), 
        (x.latitude_arrival, x.longitude_arrival)).km, axis=1)
    
    merge_transform = MergeTransformer(
        X_ext=database['AirportStatistics'], 
        cols_to_rename={'Date': 'DateOfDeparture', 'AirPort': 'Departure'}, 
        how='left',
        on=['DateOfDeparture', 'Departure'])

    X = merge_transform.fit_transform(X)
    
    print("arrival statistics departure = ", X.shape)
    
    merge_transform = MergeTransformer(
        X_ext=database['AirportStatistics'], 
        cols_to_rename={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'}, 
        how='left',
        on=['DateOfDeparture', 'Arrival'])

    X = merge_transform.fit_transform(X)
        
    database['StateFeatures'].rename({'Abbreviation': 'state_departure'}, axis=1, inplace=True)    
    merge_transform = MergeTransformer(
        X_ext=database['StateFeatures'],
        cols_to_keep=['DateOfDeparture', 'state_departure', 'UnemploymentRate', 'holidays', 'GDP_per_cap'],
        how='left',
        on=['DateOfDeparture', 'state_departure'])
    
    X = merge_transform.fit_transform(X)
    
    print("State features = ", X.shape)
        
    return X

In [4]:
database = create_db()

KeyError: "None of [Index(['name', 'pop2010'], dtype='object')] are in the [columns]"

In [ ]:
print(X_train.info())
X_train = merge_dfs(database, X_train)

In [ ]:
X_test = merge_dfs(database, X_test)

In [ ]:
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)

In [ ]:
y_train_pd = pd.Series(y_train)
y_test_pd = pd.Series(y_test)

y_train_pd.to_csv('../data/y_train.csv', index=False)
y_test_pd.to_csv('../data/y_test.csv', index=False)